In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import numpy as np
import random
from abc import ABC,abstractmethod

import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

# Any results you write to the current directory are saved as output.


In [2]:
class OUActionNoise:
    def __init__(self, mean, std_deviation, theta=0.15, dt=1e-2, x_initial=None):
        self.theta = theta
        self.mean = mean
        self.std_dev = std_deviation
        self.dt = dt
        self.x_initial = x_initial
        self.reset()

    def __call__(self):
        # Formula taken from https://www.wikipedia.org/wiki/Ornstein-Uhlenbeck_process.
        x = (
            self.x_prev
            + self.theta * (self.mean - self.x_prev) * self.dt
            + self.std_dev * np.sqrt(self.dt) * np.random.normal(size=self.mean.shape)
        )
        # Store x into x_prev
        # Makes next noise dependent on current one
        self.x_prev = x
        return x

    def reset(self):
        if self.x_initial is not None:
            self.x_prev = self.x_initial
        else:
            self.x_prev = np.zeros_like(self.mean)
            
def comupute_cumulative_feed(feed_profile):
    total_feed = 0
    # if isMDP:
    #     total_feed = 0
    #     for i in range(feed_BN.shape[0] - 1):
    #         total_feed += (feed_BN.iloc[i + 1][0] - feed_BN.iloc[i][0]) * (feed_BN.iloc[i][1])
    # else:
    for i, feed_rate in enumerate(feed_profile[1,:-1]):
        total_feed += feed_rate * (feed_profile[0,i+1] - feed_profile[0,i])
    return total_feed

def compute_reward(cumulative_feed, state, horizon):
    titer = state[1]
    total_CA = state[1] * state[-1]
    oil_consumed = cumulative_feed * 917 - state[2] * state[-1]
    yield_CA = total_CA / oil_consumed
    pv = titer / horizon
    man_cost = (2 / pv + 1 / yield_CA) / 1000  # $/(g h)
    total_reward = - man_cost * horizon  # * state[-1] * state[1]
    return total_reward

def compute_avg_reward(bn, params, policy, time_measurement,  real_measurement, B, isMDP=True):
    reward = np.zeros(B)
    for b in tqdm(range(B)):
        x0 = bn.initial_state_full + np.abs(
            np.random.normal(0, np.array(bn.initial_state_full) / 10 + 0.01))
        # time_measurement = [4 * i for i in range(bn.n_time)]
        # time_measurement[-1] = 135.9
        simulator = bioprocess_simulator(bn, time_measurement, policy, real_measurement)
        states = simulator.g(time_measurement, x0, params)
        feed = pd.DataFrame(simulator.feed_every_second, columns=['time', 'feed_rate'],
                               dtype="float64").drop_duplicates().to_numpy() if isMDP else np.vstack([real_measurement, np.array(policy.measurement)]).T
        total_feed = comupute_cumulative_feed(feed)
        reward[b] = compute_reward(total_feed, states[-1,:], time_measurement[-1])
    return np.mean(reward)

class BaseSimulator(ABC):
    @abstractmethod
    def f(self, xs, t, ps):
        pass

    @abstractmethod
    def g(self, xs, t, ps):
        pass

class base_gradient(ABC):
    @abstractmethod
    def func(self, theta):
        pass

    @abstractmethod
    def grad_f(self, theta):
        pass
    
from collections import deque

class LineSearchOptimizer(object):
    def __init__(self, f, grad, step_size, memory_size=1, **kwargs):
        self.convergence = []
        self.convergence_f = []
        self._f = f
        self._grad = grad
        if step_size is not None:
            step_size.assign_function(f, grad, self._f_update_x_next)
        self._step_size = step_size
        self._par = kwargs
        self._grad_mem = deque(maxlen=memory_size)
        self.patience = 15
        self.window = 15
        
    def get_convergence(self):
        return self.convergence
    
    def solve(self, x0, max_iter=100, tol=1e-6, disp=False):
        self.convergence = []
        self.convergence_f = []
        self._x_current = x0.copy()
        self.convergence.append(self._x_current)
        self.convergence_f.append(self._f(self._x_current))
        iteration = 0
        while True:
            h = self.get_direction(self._x_current)
            self._grad_mem.append(h)
            self._alpha = self.get_stepsize()
            self._update_x_next()
            self._update_x_current()
            self._append_conv()
            iteration += 1
            if disp > 1:
                fx = self._f(self._x_current)
                self._append_conv_f(fx)
                print("Iteration {}/{}".format(iteration, max_iter))
                print("Current function val =", fx)
                self._print_info()
            if self.check_convergence(tol):
                break
            if iteration >= max_iter:
                print("Maximum iteration exceeds!")
                break
        if disp:
            print("Convergence in {} iterations".format(iteration))
            print("Function value = {}".format(self._f(self._x_current)))
            self._print_info()
        return self._get_result_x()
    
    def get_direction(self, x):
        raise NotImplementedError("You have to provide method for finding direction!")
        
    def _update_x_current(self):
        self._x_current = self._x_next
        
    def _update_x_next(self):
        self._x_next = self._f_update_x_next(self._x_current, self._alpha, self._grad_mem[-1])
        
    def _f_update_x_next(self, x, alpha, h):
        return x + alpha * h
        
    def check_convergence(self, tol):
        return np.linalg.norm(self._grad(self.convergence[-1]) - self._grad(self.convergence[-2]), ord=1) < tol
        
    def get_stepsize(self):
        raise NotImplementedError("You have to provide method for finding step size!")
    
    def _print_info(self):
        print("Norm of gradient = {}".format(np.linalg.norm(self._grad(self._x_current))))
    
    def _append_conv(self):
        self.convergence.append(self._x_next)

    def _append_conv_f(self, fx):
        self.convergence_f.append(fx)

    def _get_result_x(self):
        if self.patience > 0:
            min_index = np.argmax(self.convergence_f)
            return self.convergence[min_index]
        return self._x_current
    
class ProjectedGD(LineSearchOptimizer):
    '''
    Class represents projected gradient method
    '''
    
    def __init__(self, f, grad, projector, step_size):
        super().__init__(f, grad, step_size)
        self._projector = projector
        
    def get_direction(self, x):
        return -self._grad(x)
    
    def _f_update_x_next(self, x, alpha, h):
        return self._projector(x + alpha * h)
    
    def check_convergence(self, tol):
        if self._f(self.convergence[-2]) - self._f(self.convergence[-1]) < tol:
            return True
        else:
            return False
        
    def get_stepsize(self):
        return self._step_size.get_stepsize(self._grad_mem[-1], self.convergence[-1], len(self.convergence))
    
    def _print_info(self):
        print("Difference in function values = {}".format(self._f(self.convergence[-2]) - self._f(self.convergence[-1])))
        print("Difference in argument = {}".format(np.linalg.norm(self.convergence[-1] - self.convergence[-2])))


class ProjectedGDWithPosterior(LineSearchOptimizer):
    '''
    Class represents projected gradient method
    '''

    def __init__(self, f, grad, projector, step_size, bn_posterior, sample_var, B=100):
        super().__init__(f, grad, step_size)
        self._projector = projector
        self.posterior = bn_posterior
        self.B = B # bootstrapping
        self.sd = sample_var if len(sample_var.shape) == 1 else sample_var.to_numpy().flatten()

    def construct_bn_gradient(self, bn):
        time_measurement = [4 * i for i in range(bn.n_time)]
        flag_estimate_reward_fun = True
        m = [0] * 35 + [-15]  # $/g
        b = np.transpose(np.array([[-0.1 / 1000] * 35]))  # $/g
        c = np.zeros(shape=(36, 5))
        c[35, 1] = 1.28739 # 0.3  # product revenue
        c[35, 0] = 0 #-0.02  # purification cost
        c[35, 2] = 0 #-0.02  # purification cost
        gradient = BN_gradient(m, b, c, bn, time_measurement, flag_estimate_reward_fun)
        return gradient

    def get_direction(self, x):
        p_beta, p_v2, mu = self.posterior.posterior_sample(self.B, useFixed=False)
        sum_bstrap_grad = np.zeros(shape=x.shape)
        for b in range(self.B):
            beta = p_beta[:,:,b] # np.apply_along_axis(np.mean, 2, p_beta)
            v2 = p_v2[:,b] # np.apply_along_axis(np.mean, 1, p_v2)
            bn = bayesian_network(mu, beta, v2, 1, 5, 36, True, mu, self.sd)
            gradient = self.construct_bn_gradient(bn)
            sum_bstrap_grad += gradient.grad_f(x)

        avg_grad = sum_bstrap_grad / self.B

        return -avg_grad

    def _f_update_x_next(self, x, alpha, h):
        return self._projector(x + alpha * h)

    def check_convergence(self, tol):
        if self._f(self.convergence[-2]) - self._f(self.convergence[-1]) < tol:
            return True
        else:
            return False

    def get_stepsize(self):
        return self._step_size.get_stepsize(self._grad_mem[-1], self.convergence[-1], len(self.convergence))

    def _print_info(self):
        print(
            "Difference in function values = {}".format(self._f(self.convergence[-2]) - self._f(self.convergence[-1])))
        print("Difference in argument = {}".format(np.linalg.norm(self.convergence[-1] - self.convergence[-2])))


class ProjectedSGD(LineSearchOptimizer):
    '''
    Class represents projected gradient method
    '''

    def __init__(self, f, grad, projector, step_size, bn_posterior, sample_var, env, batch_size=8, B=50):
        super().__init__(f, grad, step_size)
        self._projector = projector
        self.env = env
        self.posterior = bn_posterior
        self.batch_size = batch_size
        self.sd = sample_var
        self.B = B

    def _yield_batches(self, x):
        theta = x.reshape((self.posterior.n_time - 1, self.posterior.num_state, self.posterior.num_action))
        std_dev = 0.005
        ou_noise = OUActionNoise(mean=np.zeros(1), std_deviation=float(std_dev) * np.ones(1))
        policy = Policy(theta, True, ou_noise)
        return self.env.generate_trajectories(policy, self.batch_size)

    def construct_bn_gradient(self, bn):
        time_measurement = [4 * i for i in range(bn.n_time)]
        flag_estimate_reward_fun = False
        # m = [-0.5] * 35 + [0]  # $/g
        # b = np.transpose(np.array([[-0.5 / 1000] * 35]))  # $/g
        # c = np.zeros(shape=(36, 5))
        # c[35, 1] = 0.3  # product revenue
        # c[35, 0] = -0.02  # purification cost
        # c[35, 2] = -0.02  # purification cost
        m = [0] * 35 + [-15]  # $/g
        b = np.transpose(np.array([[-0.1 * 1000 * 4] * 35]))  # $/g
        c = np.zeros(shape=(36, 5))
        c[35, 1] = 1.28739  # product revenue
        c[35, 0] = 0  # purification cost
        c[35, 2] = 0  # purification cost
        gradient = BN_gradient(m, b, c, bn, time_measurement, flag_estimate_reward_fun)
        return gradient

    def get_direction(self, x):
        data, total_rewards = self._yield_batches(x)
        p_beta, p_v2, mu = self.posterior.posterior_sample(self.B, useFixed=False)
        norm_data = (data - mu) / self.sd
        sum_bstrap_grad = np.zeros(shape=x.shape)
        for b in range(self.B):
            beta = p_beta[:,:,b] # np.apply_along_axis(np.mean, 2, p_beta)
            v2 = p_v2[:,b] # np.apply_along_axis(np.mean, 1, p_v2)
            bn = bayesian_network(mu, beta, v2, 1, 5, 36, True, mu, self.sd)
            gradient = self.construct_bn_gradient(bn)
            grad = gradient.grad_sgd(x, norm_data, total_rewards)
            sum_bstrap_grad += grad

        avg_grad = sum_bstrap_grad / self.B

        return -avg_grad


        return -self._grad(x, batch)

    def _f_update_x_next(self, x, alpha, h):
        return self._projector(x + alpha * h)

    def check_convergence(self, tol):
        if len(self.convergence_f) <= self.patience:
            return False
        if self.convergence_f[-1] > np.min(self.convergence_f[-self.window:]) and self.patience <= 0:
            return True
        else:
            self.patience -= 1
            return False
        # if self._f(self.convergence_f[-2]) - self._f(self.convergence_f[-1]) < tol:
        #     return True
        # else:
        #     return False

    def get_stepsize(self):
        return self._step_size.get_stepsize(self._grad_mem[-1], self.convergence[-1], len(self.convergence))

    def _print_info(self):
        print("Difference in function values = {}".format(self._f(self.convergence[-2]) - self._f(self.convergence[-1])))
        print("Difference in argument = {}".format(np.linalg.norm(self.convergence[-1] - self.convergence[-2])))
        
class kinetic_object:
    ''' objective as transition model for RL
    '''

    def __init__(self,ps, measurement, harvest_time=140):
        self.alpha_L = ps['alpha_L'].value
        self.c_max = ps['c_max'].value
        self.K_iN = ps['K_iN'].value
        self.K_iS = ps['K_iS'].value
        self.K_iX = ps['K_iX'].value
        self.K_N = ps['K_N'].value
        self.K_O = ps['K_O'].value
        self.K_S = ps['K_S'].value
        self.K_SL = ps['K_SL'].value
        self.m_s = ps['m_s'].value
        self.r_L = ps['r_L'].value
        #    V_evap = ps['V_evap'].value
        self.V_evap = ps['V_evap'].value
        self.Y_cs = ps['Y_cs'].value
        self.Y_ls = ps['Y_ls'].value
        self.Y_xn = ps['Y_xn'].value
        self.Y_xs = ps['Y_xs'].value

        self.beta_LC_max = ps['beta_LC_max'].value
        self.mu_max = ps['mu_max'].value
        # oil concentration in oil feed
        self.S_F = 917
        self.measurement = measurement
        self.harvest_time = harvest_time
        self.dt = 4

    def update_coefficients(self, X_f, S, N, O, C, L, F_S, V):
        self.beta_LC = self.K_iN / (self.K_iN + N) * S / (S + self.K_S) * self.K_iS / (self.K_iS + S) * O / (self.K_O + O) * self.K_iX / (self.K_iX + X_f) * (
                    1 - C / self.c_max) * self.beta_LC_max
        self.q_C = 2 * (1 - self.r_L) * self.beta_LC
        self.beta_L = self.r_L * self.beta_LC - self.K_SL * L / (L + X_f) * O / (O + self.K_O)
        self.mu = self.mu_max * S / (S + self.K_S) * self.K_iS / (self.K_iS + S) * N / (self.K_N + N) * O / (self.K_O + O) / (1 + X_f / self.K_iX)
        self.q_S = 1 / self.Y_xs * self.mu + O / (O + self.K_O) * S / (self.K_S + S) * self.m_s + 1 / self.Y_cs * self.q_C + 1 / self.Y_ls * self.beta_L
        self.F_B = V / 1000 * (7.14 / self.Y_xn * self.mu * X_f + 1.59 * self.q_C * X_f)
        self.D = (self.F_B + F_S) / V

    def update_time_interval(self, new_dt):
        self.dt = new_dt

    def lipid_free_cell_growth(self, X_f, V):
        dX_f = self.dt * (self.mu * X_f - (self.D - self.V_evap / V) * X_f)
        return dX_f + X_f

    def citrate_accumulation(self, X_f, C, V):
        dC = self.dt * (self.q_C * X_f - (self.D - self.V_evap / V) * C)
        return dC + C

    def lipid_accumulation(self, X_f, L, V):
        dL = self.dt * ((self.alpha_L * self.mu + self.beta_L) * X_f - (self.D - self.V_evap / V) * L)
        return dL + L

    def glucose_consumption(self, X_f, S, F_S, V):
        dS = - self.dt * (self.q_S * X_f - F_S / V * self.S_F + (self.D - self.V_evap / V) * S)
        return dS + S

    def nitrogen_consumption(self, X_f, N, V):
        dN = - self.dt * (1 / self.Y_xn * self.mu * X_f + (self.D - self.V_evap / V) * N)
        return max(dN + N, 0)

    def volume_change(self, F_S, V):
        dV = self.dt * (self.F_B + F_S - self.V_evap)
        return dV + V

    def one_step_predict(self, X_f, C, L, S, N, V, F_S, O):
        self.update_coefficients(X_f, S, N, O, C, L, F_S, V)
        # self.update_time_interval(time)
        next_X_f = max(self.lipid_free_cell_growth(X_f, V), 0)
        next_C = max(self.citrate_accumulation(X_f, C, V), 0)
        next_L = max(self.lipid_accumulation(X_f, L, V), 0)
        next_S = max(self.glucose_consumption(X_f, S, F_S, V), 0)
        next_N = max(self.nitrogen_consumption(X_f, N, V), 0)
        next_V = max(self.volume_change(F_S, V), 0)
        return [next_X_f, next_C, next_L, next_S, next_N, next_V]

    def predict(self, X_f, C, L, S, N, V, F_S, O, steps_ahead):
        out = self.one_step_predict(X_f, C, L, S, N, V, F_S[0], O)
        for i in range(1, steps_ahead):
            out = self.one_step_predict(out[0], out[1], out[2], out[3], out[4], out[5], F_S[i], O)
        return out


# out = process_model.one_step_predict(0.04, 0, 0, 30, 5, 0.6, 0, 98)
# process_model.predict(0.03, 0, 0, 10, 3, 0.7, [0]*5 + [20 / 1000]* 20 +  [5 / 1000]* 10,32,30)
#
# process_model = kinetic_object()

class fermentation_env():
    ''' fermentation simulation environment
    '''

    def __init__(self, obj, bn, time_measurement, real_measurement, ps=None, initial_state = [0.05, 0, 0, 30, 5, 0.7]):
        self.done = False
        self.initial_state = initial_state
        self.state = initial_state
        self.simulator = obj
        self.state_indices = [0,1,3,4,5]
        self.bn = bn
        self.ps = ps
        self.time_measurement = time_measurement
        self.real_measurement = real_measurement

    def reset(self):
        self.done = False
        self.state = self.initial_state + [0]
        self.glucose_consumed = 0
        self.glucose_added = self.initial_state[3]
        return self.state

    def cost(self, weight_yield, productivity):
        return (1 / weight_yield + 2 / productivity) / 1000

    def dissolve_oxygen(self, t):
        oxygen_data = [98.58750, 71.34750, 43.57285, 28.51669, 24.61062, 24.58139, 28.96252, 30.63478, 24.34490,
                       29.33842, 31.89646, 34.51994, 36.03211, 36.16354]

        index = 0
        time_w = -1
        for i, time in enumerate(self.simulator.measurement):
            if time - t > 0:
                index = i - 1
                time_w = time
                break
        return oxygen_data[index]

    def step(self, state, action, t):
        next_state = self.simulator.one_step_predict(self.state[0], self.state[1], self.state[2], self.state[3],
                                                     self.state[4], self.state[5], action, self.dissolve_oxygen(t))
        glucose_added = action * self.simulator.dt
        return next_state, glucose_added

    def generate_trajectory(self, policy):
        state = self.initial_state
        cumulative_feed = 0
        trajectory = np.array([])
        for t, time in enumerate(self.simulator.measurement[:-1]):
            partial_state = [state[i] for i in self.state_indices]
            partial_state = (partial_state - self.bn.mu[(self.bn.n_factor * (t + 1) - self.bn.num_state):(
                    self.bn.n_factor * (t + 1))]) / \
                    self.bn.sample_sd[(self.bn.n_factor * (t + 1) - self.bn.num_state):(self.bn.n_factor * (t + 1))]
            action = policy.next_action(partial_state, t)
            action = action if np.isscalar(action) else action[0] # single action
            trajectory = np.concatenate((trajectory, [action], partial_state))
            state, glucose_added = self.step(state, action, t)
            cumulative_feed += glucose_added
        action = 0
        trajectory = np.concatenate((trajectory, [action], [state[i] for i in self.state_indices]))
        titer = state[1]
        total_CA = state[1] * state[-1]
        oil_consumed = cumulative_feed * self.simulator.S_F - state[2] * state[-1]
        yield_CA = total_CA / oil_consumed
        pv = titer / self.simulator.measurement[-1]
        man_cost = (2 / pv + 1 / yield_CA) / 1000  # $/(g h)
        total_reward = - man_cost * self.simulator.measurement[-1] * state[-1] * state[1]
        return trajectory, total_reward

    def generate_trajectories(self, policy, r):
        trajectories = []
        total_rewards = []
        while len(trajectories) < r:
            success, trajectory, total_reward = self._generate_trajectory(policy)
            if not success:
                continue
            trajectories.append(trajectory)
            total_rewards.append(total_reward)
        data = np.vstack(trajectories)
        return data, total_rewards

    def _generate_trajectory(self, policy):
        init_states = self.initial_state + np.abs(
            np.random.normal(0, np.array(self.initial_state) / 10 + 0.01))

        # exploration vs exploitation
        # if np.random.uniform(0,1) < 0.2:
        #     theta = np.random.uniform(low=0., high=0.3, size=(self.bn.n_time - 1, self.bn.num_state, self.bn.num_action))
        #     policy = Policy(theta, True)

        simulator = bioprocess_simulator(self.bn, self.time_measurement, policy, self.real_measurement)
        result = simulator.g(self.time_measurement, init_states, self.ps)
        result = result[:, self.state_indices]
        state = result[-1,:]
        trajectory = []
        for t in range(len(self.time_measurement)):
            if t not in simulator.feed.keys() and (t+1 not in simulator.feed.keys() or t-1 not in simulator.feed.keys()):
                return False, None, None
            if t not in simulator.feed.keys():
                simulator.feed[t] = (simulator.feed[t+1] + simulator.feed[t-1]) / 2
                trajectory.append((simulator.feed[t+1] + simulator.feed[t-1]) / 2)
            else:
                trajectory.append(simulator.feed[t])
            trajectory.append(result[t, 0])
            trajectory.append(result[t, 1])
            trajectory.append(result[t, 2])
            trajectory.append(result[t, 3])
            trajectory.append(result[t, 4])
            # trajectory = np.concatenate((trajectory, simulator.feed[t], result[t,:]))

        cumulative_feed = 0
        for t, time in enumerate(simulator.time_measurement[:-1]):
            cumulative_feed += simulator.feed[t] * (simulator.time_measurement[t+1] - simulator.time_measurement[t])

        total_reward = self.compute_reward(cumulative_feed, state)
        return True, np.array(trajectory), total_reward

    def compute_reward(self, cumulative_feed, state):
        titer = state[1]
        total_CA = state[1] * state[-1]
        oil_consumed = cumulative_feed * 917 - state[2] * state[-1]
        yield_CA = total_CA / oil_consumed
        pv = titer / self.time_measurement[-1]
        man_cost = (2 / pv + 1 / yield_CA) / 1000  # $/(g h)
        total_reward = - man_cost * self.time_measurement[-1] # * state[-1] * state[1]
        return total_reward
    

class fermentation_env2():
    ''' fermentation simulation environment
    '''

    def __init__(self, obj, initial_state=[0.05, 0, 0, 30, 5, 0.7]):
        self.done = False
        self.initial_state = initial_state
        self.state = initial_state + [0]
        self.obj = obj
        self.glucose_consumed = 0
        self.glucose_added = 0  # self.initial_state[3]
        self.b = -0.13363 * 1000 * 4
    def reset(self):
        self.done = False
        self.state = self.initial_state + [0]
        self.glucose_consumed = 0
        self.glucose_added = 0  # self.initial_state[3]
        return self.state

    def compute_reward(self, cumulative_feed, state):
        titer = state[1]
        total_CA = state[1] * state[-1]
        oil_consumed = cumulative_feed * 917 + self.initial_state[3] * state[-1] - state[3] * state[-1]
        yield_CA = total_CA / oil_consumed
        pv = titer / self.obj.harvest_time
        man_cost = (2 / pv + 1 / yield_CA) / 1000  # $/(g h)
        total_reward = - man_cost * self.obj.harvest_time  # * state[-1] * state[1]
        return total_reward

    def step(self, action):
        next_state = self.obj.predict(self.state[0], self.state[1], self.state[2], self.state[3], self.state[4],
                                      self.state[5], action[0], self.state[-1] * 4) + [self.state[-1] + 1]
        self.glucose_added += action[0] * self.obj.delta_t
        self.done = True if self.obj.harvest_time / self.obj.delta_t + 1 == next_state[-1] else False
        reward = self.b * action[0] + 1.28739 * (next_state[1] - self.state[1])
        # if self.done:
        #     reward = self.compute_reward(self.glucose_added - action[0], self.state[:-1])
        # else:
        #     reward = 0

        self.state = next_state
        return next_state, reward, self.done

In [3]:

class OUActionNoise:
    def __init__(self, mean, std_deviation, theta=0.15, dt=1e-2, x_initial=None):
        self.theta = theta
        self.mean = mean
        self.std_dev = std_deviation
        self.dt = dt
        self.x_initial = x_initial
        self.reset()

    def __call__(self):
        # Formula taken from https://www.wikipedia.org/wiki/Ornstein-Uhlenbeck_process.
        x = (
            self.x_prev
            + self.theta * (self.mean - self.x_prev) * self.dt
            + self.std_dev * np.sqrt(self.dt) * np.random.normal(size=self.mean.shape)
        )
        # Store x into x_prev
        # Makes next noise dependent on current one
        self.x_prev = x
        return x

    def reset(self):
        if self.x_initial is not None:
            self.x_prev = self.x_initial
        else:
            self.x_prev = np.zeros_like(self.mean)


class Buffer:
    def __init__(self, buffer_capacity=40000, batch_size=64):

        # Number of "experiences" to store at max
        self.buffer_capacity = buffer_capacity
        # Num of tuples to train on.
        self.batch_size = batch_size

        # Its tells us num of times record() was called.
        self.buffer_counter = 0

        # Instead of list of tuples as the exp.replay concept go
        # We use different np.arrays for each tuple element
        self.state_buffer = np.zeros((self.buffer_capacity, num_states))
        self.action_buffer = np.zeros((self.buffer_capacity, num_actions))
        self.reward_buffer = np.zeros((self.buffer_capacity, 1))
        self.next_state_buffer = np.zeros((self.buffer_capacity, num_states))

    # Takes (s,a,r,s') obervation tuple as input
    def record(self, obs_tuple):
        # Set index to zero if buffer_capacity is exceeded,
        # replacing old records
        index = self.buffer_counter % self.buffer_capacity

        self.state_buffer[index] = obs_tuple[0]
        self.action_buffer[index] = obs_tuple[1]
        self.reward_buffer[index] = obs_tuple[2]
        self.next_state_buffer[index] = obs_tuple[3]

        self.buffer_counter += 1

    # We compute the loss and update parameters
    def learn(self):
        # Get sampling range
        record_range = min(self.buffer_counter, self.buffer_capacity)
        # Randomly sample indices
        batch_indices = np.random.choice(record_range, self.batch_size)

        # Convert to tensors
        state_batch = tf.convert_to_tensor(self.state_buffer[batch_indices])
        action_batch = tf.convert_to_tensor(self.action_buffer[batch_indices])
        reward_batch = tf.convert_to_tensor(self.reward_buffer[batch_indices])
        reward_batch = tf.cast(reward_batch, dtype=tf.float32)
        next_state_batch = tf.convert_to_tensor(self.next_state_buffer[batch_indices])

        # Training and updating Actor & Critic networks.
        # See Pseudo Code.
        with tf.GradientTape() as tape:
            target_actions = target_actor(next_state_batch)
            y = reward_batch + gamma * target_critic([next_state_batch, target_actions])
            critic_value = critic_model([state_batch, action_batch])
            critic_loss = tf.math.reduce_mean(tf.math.square(y - critic_value))

        critic_grad = tape.gradient(critic_loss, critic_model.trainable_variables)
        critic_optimizer.apply_gradients(
            zip(critic_grad, critic_model.trainable_variables)
        )

        with tf.GradientTape() as tape:
            actions = actor_model(state_batch)
            critic_value = critic_model([state_batch, actions])
            # Used `-value` as we want to maximize the value given
            # by the critic for our actions
            actor_loss = -tf.math.reduce_mean(critic_value)

        actor_grad = tape.gradient(actor_loss, actor_model.trainable_variables)
        actor_optimizer.apply_gradients(
            zip(actor_grad, actor_model.trainable_variables)
        )


# This update target parameters slowly
# Based on rate `tau`, which is much less than one.
def update_target(tau):
    new_weights = []
    target_variables = target_critic.weights
    for i, variable in enumerate(critic_model.weights):
        new_weights.append(variable * tau + target_variables[i] * (1 - tau))

    target_critic.set_weights(new_weights)

    new_weights = []
    target_variables = target_actor.weights
    for i, variable in enumerate(actor_model.weights):
        new_weights.append(variable * tau + target_variables[i] * (1 - tau))

    target_actor.set_weights(new_weights)


def get_actor():
    # Initialize weights between -3e-3 and 3-e3
    last_init = tf.random_uniform_initializer(minval=-0.1, maxval=0.1)

    inputs = layers.Input(shape=(num_states,))
    out = layers.Dense(16, activation="relu", kernel_initializer=last_init)(inputs)
    out = layers.Dropout(0.3)(out)
    # out = layers.BatchNormalization()(out)
    # out = layers.Dense(128, activation="relu")(out)
    # out = layers.BatchNormalization()(out)
    outputs = layers.Dense(1, activation="tanh", kernel_initializer=last_init)(out)

    # Our upper bound is 2.0 for Pendulum.
    outputs = outputs * upper_bound
    model = tf.keras.Model(inputs, outputs)
    return model


def get_critic():
    # State as input
    state_input = layers.Input(shape=(num_states))
    state_out = layers.Dense(64, activation="relu")(state_input)
    # state_out = layers.BatchNormalization()(state_out)

    # Action as input
    action_input = layers.Input(shape=(num_actions))
    # action_out = layers.Dense(32, activation="relu")(action_input)
    # action_out = layers.BatchNormalization()(action_out)

    # Both are passed through seperate layer before concatenating
    concat = layers.Concatenate()([state_out, action_input])

    out = layers.Dense(8, activation="tanh")(concat)
    out = layers.Dropout(0.3)(out)
    # out = layers.BatchNormalization()(out)
    # out = layers.Dense(128, activation="relu")(out)
    # out = layers.BatchNormalization()(out)
    outputs = layers.Dense(1)(out)

    # Outputs single value for give state-action
    model = tf.keras.Model([state_input, action_input], outputs)

    return model



class mechanism:
    ''' mechanism model as dynamics for RL
    '''

    def __init__(self, harvest_time=140):

        self.alpha_L = 0.127275
        self.c_max = 130.901733
        self.K_iN = 0.12294103
        self.K_iS = 612.178130
        self.K_iX = 59.9737695
        self.K_N = 0.02000201
        self.K_O = 0.33085322
        self.K_S = 0.0430429
        self.K_SL = 0.02165744
        self.m_s = 0.02252332
        self.r_L = 0.47917813
        self.V_evap = 2.6 * 1e-03
        self.Y_cs = 0.682572
        self.Y_ls = 0.3574429
        self.Y_xn = 10.0
        self.Y_xs = 0.2385559
        self.beta_LC_max = 0.14255192
        self.mu_max = 0.3844627
        # set up the kinetic constants
        self.dt = 0.001  # control in every hour
        # control variable
        # F_B
        # glucose concentration in glucose feed
        self.S_F = 917
        self.harvest_time = harvest_time
        self.delta_t = 4
        self.real_measurement = [0.0, 5.0, 10.0, 23.0, 28.0, 34.0, 47.5, 55.0, 72.0, 80.0, 95.0, 102.0, 120.0, 140.0]

    def update_coefficients(self, X_f, S, N, O, C, L, F_S, V):

        self.beta_LC = self.K_iN / (self.K_iN + N) * S / (S + self.K_S) * self.K_iS / (self.K_iS + S) * O / (
                    self.K_O + O) * self.K_iX / (self.K_iX + X_f) * (1 - C / self.c_max) * self.beta_LC_max
        self.q_C = 2 * (1 - self.r_L) * self.beta_LC
        self.beta_L = self.r_L * self.beta_LC - self.K_SL * L / (L + X_f) * O / (O + self.K_O)
        self.mu = self.mu_max * S / (S + self.K_S) * self.K_iS / (self.K_iS + S) * N / (self.K_N + N) * O / (
                    self.K_O + O) / (1 + X_f / self.K_iX)
        self.q_S = 1 / self.Y_xs * self.mu + O / (O + self.K_O) * S / (
                    self.K_S + S) * self.m_s + 1 / self.Y_cs * self.q_C + 1 / self.Y_ls * self.beta_L
        self.F_B = V / 1000 * (7.14 / self.Y_xn * self.mu * X_f + 1.59 * self.q_C * X_f)
        self.D = (self.F_B + F_S) / V

    def lipid_free_cell_growth(self, X_f, V):
        dX_f = self.dt * (self.mu * X_f - (self.D - self.V_evap / V) * X_f)
        return dX_f + X_f

    def citrate_accumulation(self, X_f, C, V):
        dC = self.dt * (self.q_C * X_f - (self.D - self.V_evap / V) * C)
        return dC + C

    def lipid_accumulation(self, X_f, L, V):
        dL = self.dt * ((self.alpha_L * self.mu + self.beta_L) * X_f - (self.D - self.V_evap / V) * L)
        return dL + L

    def glucose_consumption(self, X_f, S, F_S, V):
        dS = - self.dt * (self.q_S * X_f - F_S / V * self.S_F + (self.D - self.V_evap / V) * S)
        return dS + S

    def nitrogen_consumption(self, X_f, N, V):
        dN = - self.dt * (1 / self.Y_xn * self.mu * X_f + (self.D - self.V_evap / V) * N)
        return max(dN + N, 0)

    def volume_change(self, F_S, V):
        dV = self.dt * (self.F_B + F_S - self.V_evap)
        return dV + V

    def one_step_predict(self, X_f, C, L, S, N, V, F_S, O):
        next_X_f = X_f
        next_C = C
        next_L = L
        next_S = S
        next_N = N
        next_V = V
        for i in range(int(self.delta_t / self.dt)):
            self.update_coefficients(next_X_f, next_S, next_N, O, next_C, next_L, F_S, next_V)
            next_X_f = max(self.lipid_free_cell_growth(next_X_f, next_V), 0)
            next_C = max(self.citrate_accumulation(next_X_f, next_C, next_V), 0)
            next_L = max(self.lipid_accumulation(next_X_f, next_L, next_V), 0)
            next_S = max(self.glucose_consumption(next_X_f, next_S, F_S, next_V), 0)
            next_N = max(self.nitrogen_consumption(next_X_f, next_N, next_V), 0)
            next_V = max(self.volume_change(F_S, next_V), 0)
        return [next_X_f, next_C, next_L, next_S, next_N, next_V]

    def predict(self, X_f, C, L, S, N, V, F_S, t):
        O = self.dissolve_oxygen(t)
        out = self.one_step_predict(X_f, C, L, S, N, V, F_S, O)
        return out

    def dissolve_oxygen(self, t):
        oxygen_data = [98.58750, 71.34750, 43.57285, 28.51669, 24.61062, 24.58139, 28.96252, 30.63478, 24.34490,
                       29.33842, 31.89646, 34.51994, 36.03211, 36.16354]

        index = 0
        time_w = -1
        for i, time in enumerate(self.real_measurement):
            if time - t > 0:
                index = i - 1
                time_w = time
                break
        return oxygen_data[index]


# out = process_model.one_step_predict(0.04, 0, 0, 30, 5, 0.6, 0, 98)
# process_model.predict(0.03, 0, 0, 10, 3, 0.7, [0]*5 + [20 / 1000]* 20 +  [5 / 1000]* 10,32,30)

# process_model = kinetic_object()


In [4]:
macro_rep = 10
np.random.seed(2001)
avg_reward_lists = []
episodic_reward_lists = []
for m in range(macro_rep):
    problem = "Feed-control-v0"
    process_model = mechanism()
    env = fermentation_env2(process_model)

    num_states = 7 # env.observation_space.shape[0]
    print("Size of State Space ->  {}".format(num_states))
    num_actions = 1 # env.action_space.shape[0]
    print("Size of Action Space ->  {}".format(num_actions))

    upper_bound = 0.02 # upper bounad of feeding
    lower_bound = 0.0 # upper bounad of feeding

    def policy(state, noise_object):
        sampled_actions = tf.squeeze(actor_model(state))
        noise = noise_object()
        # Adding noise to action
        sampled_actions = sampled_actions.numpy() + noise

        # We make sure action is within bounds
        legal_action = np.clip(sampled_actions, lower_bound, upper_bound)

        return [np.squeeze(legal_action)]

    std_dev = 0.0001
    ou_noise = OUActionNoise(mean=np.zeros(1), std_deviation=float(std_dev) * np.ones(1))

    actor_model = get_actor()
    critic_model = get_critic()

    target_actor = get_actor()
    target_critic = get_critic()

    # Making the weights equal initially
    target_actor.set_weights(actor_model.get_weights())
    target_critic.set_weights(critic_model.get_weights())

    # Learning rate for actor-critic models
    critic_lr = 0.0001
    actor_lr = 0.0001

    critic_optimizer = tf.keras.optimizers.Adam(critic_lr)
    actor_optimizer = tf.keras.optimizers.Adam(actor_lr)

    # Discount factor for future rewards
    gamma = 1
    # Used to update target networks
    tau = 0.00

    buffer = Buffer(50000, 64)


    # To store reward history of each episode
    ep_reward_list = []
    # To store average reward history of last few episodes
    avg_reward_list = []

    total_episodes = 400

    avg_reward_best = 0

    # Takes about 20 min to train
    for ep in range(total_episodes):

        prev_state = env.reset()
        episodic_reward = 0
        length_ep = 0
        while True:
            # Uncomment this to see the Actor in action
            # But not in a python notebook.
            # env.render()
            tf_prev_state = tf.expand_dims(tf.convert_to_tensor(prev_state), 0)
            action = policy(tf_prev_state, ou_noise)
            # disturb training when it's stuck at 0 feeding rate
            if len(avg_reward_list) > 3 and abs(avg_reward_list[-2] - avg_reward_list[-1]) + abs(
                    avg_reward_list[-3] - avg_reward_list[-1]) < 1e-10:
                action = [np.squeeze([0.003])]

            # Recieve state and reward from environment.
            # action[0] = 0.01
            state, reward, done = env.step(action)
            buffer.record((prev_state, action, reward, state))
            episodic_reward += reward
            buffer.learn()
            update_target(tau)

            # End this episode when `done` is True
            if done:
                break

            prev_state = state

        ep_reward_list.append(episodic_reward)
        # Mean of last 40 episodes
        avg_reward = np.mean(ep_reward_list[-40:])
        if avg_reward > avg_reward_best:
            avg_reward_best = avg_reward
            file_path_actor_best = './{}/best/ddpg-actor'.format(m)
            file_path_critic_best = './{}//best/ddpg-critic'.format(m)
            actor_model.save(file_path_actor_best)
            critic_model.save(file_path_critic_best)
            if avg_reward > 100:
                file_path_actor = './{0}/ddpg-actor-ep:{1}-alr:{2}-clr:{3}'.format(m, ep, actor_lr, critic_lr)
                file_path_critic = './{0}/ddpg-critic-ep:{1}-alr:{2}-clr:{3}'.format(m, ep, actor_lr, critic_lr)
                actor_model.save(file_path_actor)
                critic_model.save(file_path_critic)
        print("Episode * {} * Avg Reward is ==> {}".format(ep, avg_reward))
        avg_reward_list.append(avg_reward)

    avg_reward_lists.append(avg_reward_list)
    episodic_reward_lists.append(ep_reward_list)
    file_path_actor = './{0}/ddpg-actor-final-alr:{1}-clr:{2}'.format(m, actor_lr, critic_lr)
    file_path_critic = './{0}/ddpg-critic-final-alr:{1}-clr:{2}'.format(m, actor_lr, critic_lr)
    actor_model.save(file_path_actor)
    critic_model.save(file_path_critic)
    np.save('avg_reward_lists.npy', avg_reward_lists)
    np.save('episodic_reward_lists.npy', episodic_reward_lists)

Size of State Space ->  7
Size of Action Space ->  1
Episode * 0 * Avg Reward is ==> 51.284514944508295
Episode * 1 * Avg Reward is ==> 55.535418624986804
Episode * 2 * Avg Reward is ==> 59.96832689490301
Episode * 3 * Avg Reward is ==> 63.84209006290904
Episode * 4 * Avg Reward is ==> 66.96061387666256
Episode * 5 * Avg Reward is ==> 69.03884158248057
Episode * 6 * Avg Reward is ==> 70.24057878987682
Episode * 7 * Avg Reward is ==> 70.85542031593641
Episode * 8 * Avg Reward is ==> 70.82050578639834
Episode * 9 * Avg Reward is ==> 34.10041532481152
Episode * 10 * Avg Reward is ==> 3.1268817173456323
Episode * 11 * Avg Reward is ==> -22.860287076946832
Episode * 12 * Avg Reward is ==> -44.937769418565104
Episode * 13 * Avg Reward is ==> -63.937939704156925
Episode * 14 * Avg Reward is ==> -80.4424927534525
Episode * 15 * Avg Reward is ==> -94.90765695501914
Episode * 16 * Avg Reward is ==> -107.68769181083746
Episode * 17 * Avg Reward is ==> -119.05076757529403
Episode * 18 * Avg Reward